# Grid2WS: Spatial and Temporal Validation of Multi-Source Precipitation Products

**Author:** Song  
**Description:** This notebook processes native-grid footprint geometries (DAYMET, PRISM, gridMET) extracted via Google Earth Engine. It performs exact geometric intersection with watershed boundaries in a projected coordinate system (NAD83 / UTM Zone 17N) to calculate rigorous area-weighted precipitation means. Finally, it validates these spatial products against local meteorological station and rain gauge data.

---

In [10]:
# ====================================================================
# Cell 1 — Environment Setup & Global Configurations
# ====================================================================
import os
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from matplotlib.ticker import FuncFormatter
import seaborn as sns

warnings.filterwarnings('ignore') # Suppress harmless geopandas overlay warnings

# --------------------------------------------------------------------
# A. Scientific Publication Style Settings
# --------------------------------------------------------------------
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'font.family': 'sans-serif',
    'font.sans-serif': ['Arial', 'DejaVu Sans'],
    'font.weight': 'bold',
    'axes.labelweight': 'bold',
    'axes.titleweight': 'bold',
    'savefig.bbox': 'tight',
    'savefig.dpi': 300
})

# --------------------------------------------------------------------
# B. Coordinate Reference Systems (CRS) & Global Constants
# --------------------------------------------------------------------
CRS_UTM = "EPSG:26917"  # NAD83 / UTM Zone 17N (For exact area calculations)
CRS_WGS = "EPSG:4326"   # WGS84 (Default for GEE exports)
YEARS = (2017, 2018)
START_DATE = f'{YEARS[0]}-01-01'
END_DATE = f'{YEARS[1]}-12-31'

# --------------------------------------------------------------------
# C. File Paths (User defined)
# --------------------------------------------------------------------
# Output Directory
OUT_DIR = Path(".").resolve() / "Grid2WS_Validation_Output"
OUT_DIR.mkdir(parents=True, exist_ok=True)
print(f"Output directory initialized at: {OUT_DIR}")

# Watershed Boundaries (Exported from GEE)
BASIN_CA_GEE = r"/Users/benthosyy/Downloads/P_consistency/drive-download-20260226T002239Z-1-001/GEE_WS_CABR_export_for_CRS_audit.shp"
BASIN_AR_GEE = r"/Users/benthosyy/Downloads/P_consistency/drive-download-20260226T002239Z-1-001/GEE_WS_ARWD_export_for_CRS_audit.shp"

# Native-Grid Datasets (Footprints and Daily CSVs from GEE)
datasets_to_process = {
    "DAYMET": {
        "shp": r"/Users/benthosyy/Downloads/P_consistency/P_nativegrid_pixels_strict/DAYMET_STRICT_nativegrid_pixel_footprints_CA_AR.shp",
        "csv": r"/Users/benthosyy/Downloads/P_consistency/P_nativegrid_pixels_strict/DAYMET_STRICT_nativegrid_pixels_daily_2017_2018.csv"
    },
    "PRISM": {
        "shp": r"/Users/benthosyy/Downloads/P_consistency/P_nativegrid_pixels_strict/PRISM_STRICT_nativegrid_pixel_footprints_CA_AR.shp",
        "csv": r"/Users/benthosyy/Downloads/P_consistency/P_nativegrid_pixels_strict/PRISM_STRICT_nativegrid_pixels_daily_2017_2018.csv"
    },
    "gridMET": {
        "shp": r"/Users/benthosyy/Downloads/P_consistency/P_nativegrid_pixels_strict/GRIDMET_STRICT_nativegrid_pixel_footprints_CA_AR.shp",
        "csv": r"/Users/benthosyy/Downloads/P_consistency/P_nativegrid_pixels_strict/GRIDMET_STRICT_nativegrid_pixels_daily_2017_2018.csv"
    }
}

# Local Rain Gauges and Meteorological Data
GAUGE_UTM = {
    "TO": {"easting": 268856.402, "northing": 3895148.822},
    "UA": {"easting": 270550.147, "northing": 3896352.771}
}

MET_FILES = {
    'CS01_Input': r"/Users/benthosyy/Desktop/CodeBits/DHSVM-PNNL-2025/TestCase/1203_StormTiming/DHSVM_met_input_CS01_hourly_0411_mean_wind_interpolation.txt",
    'Gauge_TO':   r"/Users/benthosyy/Desktop/CodeBits/DHSVM-PNNL-2025/TestCase/1203_StormTiming/processed_rain_gauges_step2/DHSVM_met_input_TO_hourly_2017_2018.txt",
    'Gauge_UA':   r"/Users/benthosyy/Desktop/CodeBits/DHSVM-PNNL-2025/TestCase/1203_StormTiming/processed_rain_gauges_step2/DHSVM_met_input_UA_hourly_2017_2018.txt"
}

# Timeseries Visualization Mapping
WS_MAPPING = {
    'Camp Branch (CA)': {'csv_col': 'CA_P_mm', 'gauge_key': 'Gauge_TO'},
    'Arrowwood (AR)':   {'csv_col': 'AR_P_mm', 'gauge_key': 'Gauge_UA'}
}

Output directory initialized at: /Users/benthosyy/Desktop/NEXRAD/Grid2WS/2_local_processing/Grid2WS_Validation_Output


---
### Spatial Data Initialization
All spatial geometries are rigorously reprojected to EPSG:26917 to ensure conservation of mass when calculating geometric intersection areas.

In [11]:
# ====================================================================
# Cell 2 — Spatial Data Initialization: Basins & Rain Gauges
# ====================================================================

def read_geospatial(path, default_crs=CRS_WGS):
    """Safely loads spatial data and assigns a default CRS if missing."""
    gdf = gpd.read_file(path)
    if gdf.crs is None:
        gdf = gdf.set_crs(default_crs)
    return gdf

print("Loading and projecting basin boundaries to UTM 17N...")
# 1. Read Basin Shapefiles (WGS84 from GEE)
ca_basin = read_geospatial(BASIN_CA_GEE, default_crs=CRS_WGS)
ar_basin = read_geospatial(BASIN_AR_GEE, default_crs=CRS_WGS)

# 2. Reproject Basins to UTM 17N for planar area calculations
ca_utm = ca_basin.to_crs(CRS_UTM)
ar_utm = ar_basin.to_crs(CRS_UTM)

print("Initializing rain gauge coordinate objects...")
# 3. Create Rain Gauges GeoDataFrame
gauges_utm = gpd.GeoDataFrame(
    {
        "name": ["TO", "UA"],
        "geometry": [
            gpd.points_from_xy([GAUGE_UTM["TO"]["easting"]], [GAUGE_UTM["TO"]["northing"]])[0],
            gpd.points_from_xy([GAUGE_UTM["UA"]["easting"]], [GAUGE_UTM["UA"]["northing"]])[0]
        ],
    },
    crs=CRS_UTM
)

print("Spatial basemap initialized.")

Loading and projecting basin boundaries to UTM 17N...
Initializing rain gauge coordinate objects...
Spatial basemap initialized.


---
### Core Engine: Exact Spatial Intersection
This cell contains the primary function to overlay the native grids onto the watersheds, calculate area weights, and generate publication-quality spatial layout figures.

In [12]:
# ====================================================================
# Cell 3 — Core Processing & Plotting Function (Perfect Layout & Offset)
# ====================================================================
import warnings
warnings.filterwarnings('ignore') 

# --------------------------------------------------------------------
# 0. Plotting Style Constants
# --------------------------------------------------------------------
CA_COLOR = "#D62728"
AR_COLOR = "#1F77B4"
GRID_EDGE = "0.35"
GRID_LW = 0.9
FP_ALPHA = 0.95
BASIN_LW = 2.2
GAUGE_MS = 100     
LABEL_FS = 8.5     

# --------------------------------------------------------------------
# A. Plotting Helper Functions
# --------------------------------------------------------------------
def fmt_km(x, pos): return f"{int(round(x / 1000))}"

def add_north_arrow(ax, x=0.95, y=0.90, size=0.10):
    ax.annotate("N", xy=(x, y), xytext=(x, y - size), xycoords="axes fraction", textcoords="axes fraction",
                ha="center", va="center", fontsize=11, fontweight="bold", arrowprops=dict(arrowstyle="-|>", lw=1.1, color="black"))

def add_scale_bar(ax, length_m=2000, location=(0.04, 0.06), lw=2.8):
    xmin, xmax, ymin, ymax = ax.get_xlim()[0], ax.get_xlim()[1], ax.get_ylim()[0], ax.get_ylim()[1]
    x0, y0 = xmin + location[0] * (xmax - xmin), ymin + location[1] * (ymax - ymin)
    ax.plot([x0, x0 + length_m], [y0, y0], color="black", lw=lw, solid_capstyle="butt", zorder=30)
    ax.text(x0 + length_m/2, y0 + 0.02*(ymax-ymin), f"{int(length_m/1000)} km", ha="center", va="bottom", fontsize=10, fontweight="bold")

def set_axis_utm(ax, show_xlabel=True, show_ylabel=True):
    ax.set_aspect("equal", adjustable="box")
    ax.grid(True, linestyle=":", alpha=0.25)
    ax.xaxis.set_major_formatter(FuncFormatter(fmt_km))
    ax.yaxis.set_major_formatter(FuncFormatter(fmt_km))
    
    if show_xlabel: ax.set_xlabel("Easting (km, NAD83 / UTM 17N)", fontsize=12, fontweight="bold", labelpad=6)
    else: ax.set_xticklabels([])
    if show_ylabel: ax.set_ylabel("Northing (km, NAD83 / UTM 17N)", fontsize=12, fontweight="bold", labelpad=6)
    else: ax.set_yticklabels([])
    
    for lab in ax.get_xticklabels(): lab.set_rotation(0)
    for lab in ax.get_xticklabels(): lab.set_ha("center")

# target_aspect -> 1.75 -> 13.0x11.5
def get_padded_extent_fixed_aspect(*gdfs, target_aspect=1.75, pad_frac=0.08): 
    b = np.vstack([g.total_bounds for g in gdfs])
    minx, miny = b[:, 0].min(), b[:, 1].min()
    maxx, maxy = b[:, 2].max(), b[:, 3].max()
    
    dx, dy = maxx - minx, maxy - miny
    minx, maxx = minx - dx * pad_frac, maxx + dx * pad_frac
    miny, maxy = miny - dy * pad_frac, maxy + dy * pad_frac
    
    w, h = maxx - minx, maxy - miny
    current_aspect = w / h
    
    if current_aspect < target_aspect:
        new_w = h * target_aspect
        pad_x = (new_w - w) / 2
        minx, maxx = minx - pad_x, maxx + pad_x
    elif current_aspect > target_aspect:
        new_h = w / target_aspect
        pad_y = (new_h - h) / 2
        miny, maxy = miny - pad_y, maxy + pad_y
        
    return (minx, miny, maxx, maxy)

def add_pixel_labels(ax, fp, col):
    placed = []
    min_sep = 120.0
    for _, r in fp.iterrows():
        val = r.get(col)
        if pd.isna(val): continue
        pt = r.geometry.representative_point()
        x, y = pt.x, pt.y
        for (px, py) in placed:
            if (x - px)**2 + (y - py)**2 < (min_sep**2): x += 0.5 * min_sep; y += 0.5 * min_sep
        ax.text(x, y, f"{int(round(val))}", ha="center", va="center", fontsize=LABEL_FS, fontweight="bold",
                bbox=dict(boxstyle="round,pad=0.05", facecolor="white", alpha=0.75, edgecolor="none"), zorder=20) 
        placed.append((x, y))

def annotate_intersection_percents(ax, inter_gdf, top_n=None):
    g = inter_gdf.dropna(subset=["w_frac"]).copy()
    g = g[g["w_frac"] > 0].copy()
    if len(g) == 0: return
    if top_n is not None: g = g.sort_values("w_frac", ascending=False).head(int(top_n))
    for _, r in g.iterrows():
        pt = r.geometry.representative_point()
        if r['w_frac'] >= 0.01: 
            ax.text(pt.x, pt.y, f"{100*r['w_frac']:.1f}%", ha="center", va="center", fontsize=7.5, fontweight="bold",
                    bbox=dict(boxstyle="round,pad=0.08", facecolor="white", alpha=0.85, edgecolor="none"), zorder=25)

def area_weighted_mean_from_inter(inter: gpd.GeoDataFrame, value_col: str) -> float:
    g = inter.dropna(subset=[value_col]).copy()
    if len(g) == 0: return np.nan
    w = g.geometry.area.values
    v = g[value_col].values
    if np.sum(w) <= 0: return np.nan
    return float(np.sum(w * v) / np.sum(w))

# --------------------------------------------------------------------
# B. Main Processing & Plotting Loop Function
# --------------------------------------------------------------------
def process_and_plot_product(product_name, fp_path, csv_path, ca_utm, ar_utm, gauges_utm, out_dir):
    print(f"--- Processing {product_name} ---")
    
    fp = gpd.read_file(fp_path)
    if fp.crs is None: fp = fp.set_crs(CRS_WGS)
    fp_utm = fp.to_crs(CRS_UTM)
    fp_utm["geometry"] = fp_utm.geometry.buffer(0) 
    
    df = pd.read_csv(csv_path)
    df["Date"] = pd.to_datetime(df["Date"])
    df["Year"] = df["Date"].dt.year
    df = df[df["Year"].isin(YEARS)].copy()
    df_ann = df.groupby(["pixel_id", "Year"], as_index=False)["P_mm"].sum().rename(columns={"P_mm": "P_annual_mm"})
    
    df_wide = df_ann.pivot(index="pixel_id", columns="Year", values="P_annual_mm").reset_index()
    y0, y1 = YEARS
    col0, col1 = f"P{y0}", f"P{y1}"
    df_wide.rename(columns={y0: col0, y1: col1}, inplace=True)
    
    fp_ann_utm = fp_utm.merge(df_wide, on="pixel_id", how="left")
    
    ca_basin_use = ca_utm[["geometry"]].copy()
    ca_basin_use["geometry"] = ca_basin_use.geometry.buffer(0)
    ar_basin_use = ar_utm[["geometry"]].copy()
    ar_basin_use["geometry"] = ar_basin_use.geometry.buffer(0)
    
    ca_inter = gpd.overlay(fp_ann_utm, ca_basin_use, how="intersection", keep_geom_type=True)
    ar_inter = gpd.overlay(fp_ann_utm, ar_basin_use, how="intersection", keep_geom_type=True)
    
    ca_inter["w_frac"] = ca_inter.geometry.area / ca_utm.geometry.area.sum()
    ar_inter["w_frac"] = ar_inter.geometry.area / ar_utm.geometry.area.sum()

    ca_mean_y0 = area_weighted_mean_from_inter(ca_inter, col0)
    ca_mean_y1 = area_weighted_mean_from_inter(ca_inter, col1)
    ar_mean_y0 = area_weighted_mean_from_inter(ar_inter, col0)
    ar_mean_y1 = area_weighted_mean_from_inter(ar_inter, col1)

    vals = np.concatenate([fp_ann_utm[col0].dropna().values, fp_ann_utm[col1].dropna().values])
    vmin, vmax = float(np.min(vals)), float(np.max(vals))
    
    xmin, ymin, xmax, ymax = get_padded_extent_fixed_aspect(fp_ann_utm, ca_utm, ar_utm, target_aspect=1.75, pad_frac=0.08)
    
    ca_pt = ca_utm.geometry.iloc[0].representative_point()
    ar_pt = ar_utm.geometry.iloc[0].representative_point()

    fig = plt.figure(figsize=(13.0, 11.5))
    gs = fig.add_gridspec(nrows=3, ncols=2, height_ratios=[1, 1, 1], width_ratios=[1.0, 1.0], hspace=0.18, wspace=0.03)

    ax11, ax12 = fig.add_subplot(gs[0, 0]), fig.add_subplot(gs[0, 1])
    ax21, ax22 = fig.add_subplot(gs[1, 0]), fig.add_subplot(gs[1, 1])
    ax31, axL  = fig.add_subplot(gs[2, 0]), fig.add_subplot(gs[2, 1])

    def draw_pixel(ax, year_col, year_label, show_xlabel, show_ylabel):
        fp_ann_utm.plot(ax=ax, column=year_col, cmap="viridis", vmin=vmin, vmax=vmax, edgecolor=GRID_EDGE, linewidth=GRID_LW, alpha=FP_ALPHA, zorder=1)
        ca_utm.boundary.plot(ax=ax, color=CA_COLOR, linewidth=BASIN_LW, zorder=5)
        ar_utm.boundary.plot(ax=ax, color=AR_COLOR, linewidth=BASIN_LW, zorder=5)
        gauges_utm[gauges_utm["name"]=="TO"].plot(ax=ax, marker="^", color="gold", edgecolor="black", markersize=GAUGE_MS, zorder=10)
        gauges_utm[gauges_utm["name"]=="UA"].plot(ax=ax, marker="^", color="cyan", edgecolor="black", markersize=GAUGE_MS, zorder=10)
        add_pixel_labels(ax, fp_ann_utm, year_col)
        ax.set_title(f"Pixel-scale annual P ({year_label})", fontsize=13, fontweight="bold", pad=8)
        set_axis_utm(ax, show_xlabel=show_xlabel, show_ylabel=show_ylabel)
        ax.set_xlim(xmin, xmax); ax.set_ylim(ymin, ymax)
        add_north_arrow(ax); 
        if show_xlabel: add_scale_bar(ax, 2000)

    draw_pixel(ax11, col0, y0, show_xlabel=False, show_ylabel=True)
    draw_pixel(ax12, col1, y1, show_xlabel=False, show_ylabel=False)

    def draw_basinmean(ax, year, val_ca, val_ar, show_xlabel, show_ylabel):
        fp_ann_utm.plot(ax=ax, facecolor="0.92", edgecolor=GRID_EDGE, linewidth=0.8, alpha=0.55, zorder=1)
        ca_inter.plot(ax=ax, color=CA_COLOR, alpha=0.22, edgecolor="none", zorder=3)
        ar_inter.plot(ax=ax, color=AR_COLOR, alpha=0.22, edgecolor="none", zorder=3)
        ca_utm.boundary.plot(ax=ax, color=CA_COLOR, linewidth=BASIN_LW, zorder=5)
        ar_utm.boundary.plot(ax=ax, color=AR_COLOR, linewidth=BASIN_LW, zorder=5)
        gauges_utm[gauges_utm["name"]=="TO"].plot(ax=ax, marker="^", color="gold", edgecolor="black", markersize=GAUGE_MS, zorder=10)
        gauges_utm[gauges_utm["name"]=="UA"].plot(ax=ax, marker="^", color="cyan", edgecolor="black", markersize=GAUGE_MS, zorder=10)

        ctx, cty = ca_pt.x - 200, ca_pt.y - 1700
        atx, aty = ar_pt.x + 200, ar_pt.y + 1500

        ax.text(ctx, cty, f"CA: {val_ca:.0f} mm", ha="center", va="center", fontsize=9.5, fontweight="bold", bbox=dict(boxstyle="round,pad=0.25", facecolor="white", alpha=0.9, edgecolor=CA_COLOR, lw=1.5), zorder=30)
        ax.text(atx, aty, f"AR: {val_ar:.0f} mm", ha="center", va="center", fontsize=9.5, fontweight="bold", bbox=dict(boxstyle="round,pad=0.25", facecolor="white", alpha=0.9, edgecolor=AR_COLOR, lw=1.5), zorder=30)
        
        ax.set_title(f"Area-weighted basin mean, {year}", fontsize=13, fontweight="bold", pad=8)
        set_axis_utm(ax, show_xlabel=show_xlabel, show_ylabel=show_ylabel)
        ax.set_xlim(xmin, xmax); ax.set_ylim(ymin, ymax)
        add_north_arrow(ax); 
        if show_xlabel: add_scale_bar(ax, 2000)

    draw_basinmean(ax21, y0, ca_mean_y0, ar_mean_y0, show_xlabel=False, show_ylabel=True)
    draw_basinmean(ax22, y1, ca_mean_y1, ar_mean_y1, show_xlabel=True, show_ylabel=False) 

    fp_ann_utm.plot(ax=ax31, facecolor="0.92", edgecolor=GRID_EDGE, linewidth=0.8, alpha=0.55, zorder=1)
    ca_inter.plot(ax=ax31, color=CA_COLOR, alpha=0.22, edgecolor="none", zorder=3)
    ar_inter.plot(ax=ax31, color=AR_COLOR, alpha=0.22, edgecolor="none", zorder=3)
    ca_utm.boundary.plot(ax=ax31, color=CA_COLOR, linewidth=BASIN_LW, zorder=5)
    ar_utm.boundary.plot(ax=ax31, color=AR_COLOR, linewidth=BASIN_LW, zorder=5)
    gauges_utm[gauges_utm["name"]=="TO"].plot(ax=ax31, marker="^", color="gold", edgecolor="black", markersize=GAUGE_MS, zorder=10)
    gauges_utm[gauges_utm["name"]=="UA"].plot(ax=ax31, marker="^", color="cyan", edgecolor="black", markersize=GAUGE_MS, zorder=10)
    
    annotate_intersection_percents(ax31, ca_inter, top_n=4)
    annotate_intersection_percents(ax31, ar_inter, top_n=4)
    ax31.set_title("Intersection-area weights schematic", fontsize=13, fontweight="bold", pad=8)
    set_axis_utm(ax31, show_xlabel=True, show_ylabel=True)
    ax31.set_xlim(xmin, xmax); ax31.set_ylim(ymin, ymax)
    add_north_arrow(ax31); add_scale_bar(ax31, 2000)

    axL.axis("off")
    handles = [
        Line2D([0], [0], color=CA_COLOR, lw=BASIN_LW, label="Camp Branch (CA) boundary"),
        Line2D([0], [0], color=AR_COLOR, lw=BASIN_LW, label="Arrowwood (AR) boundary"),
        Patch(facecolor="0.92", edgecolor=GRID_EDGE, label="Native-grid footprints"),
        Patch(facecolor=CA_COLOR, edgecolor="none", alpha=0.22, label="Intersection area (CA $\\cap$ footprints)"),
        Patch(facecolor=AR_COLOR, edgecolor="none", alpha=0.22, label="Intersection area (AR $\\cap$ footprints)"),
        Line2D([0], [0], marker="^", color="w", markerfacecolor="gold", markeredgecolor="black", markersize=10, label="TO rain gauge"),
        Line2D([0], [0], marker="^", color="w", markerfacecolor="cyan", markeredgecolor="black", markersize=10, label="UA rain gauge"),
    ]
    axL.legend(handles=handles, loc="upper center", bbox_to_anchor=(0.5, 0.95), frameon=False, fontsize=11.5)
    
    cax = axL.inset_axes([0.15, 0.15, 0.70, 0.05]) 
    sm = plt.cm.ScalarMappable(cmap="viridis", norm=plt.Normalize(vmin=vmin, vmax=vmax))
    sm._A = []
    cb = fig.colorbar(sm, cax=cax, orientation="horizontal", extend="both", extendrect=False)
    cb.set_label("Annual precipitation (mm)", fontsize=12, fontweight="bold", labelpad=8)
    cb.ax.tick_params(labelsize=10)

    fig.suptitle(f"{product_name} annual precipitation and basin area-weighting (UTM; {y0}–{y1})", fontsize=17, fontweight="bold", y=0.95)
    
    png_path = out_dir / f"{product_name}_layoutB_UTM_{y0}_{y1}.png"
    pdf_path = out_dir / f"{product_name}_layoutB_UTM_{y0}_{y1}.pdf"
    fig.savefig(png_path, dpi=300, bbox_inches="tight")
    fig.savefig(pdf_path, dpi=300, bbox_inches="tight")
    print(f"[{product_name}] Finished and saved to {png_path.name}!\n")
    
    plt.close(fig)

---
### Batch Execution: Spatial Footprint Mapping
Executes the rendering loop to generate spatial validation plots for all provided gridded products.

In [13]:
# ====================================================================
# Cell 4 — Batch Execution: Spatial Footprint Mapping
# ====================================================================

for product_name, paths in datasets_to_process.items():
    process_and_plot_product(
        product_name=product_name,
        fp_path=paths["shp"],
        csv_path=paths["csv"],
        ca_utm=ca_utm,         
        ar_utm=ar_utm,         
        gauges_utm=gauges_utm, 
        out_dir=OUT_DIR
    )

print("🎉 Spatial mapping successfully finished!")

--- Processing DAYMET ---
[DAYMET] Finished and saved to DAYMET_layoutB_UTM_2017_2018.png!

--- Processing PRISM ---
[PRISM] Finished and saved to PRISM_layoutB_UTM_2017_2018.png!

--- Processing gridMET ---
[gridMET] Finished and saved to gridMET_layoutB_UTM_2017_2018.png!

🎉 Spatial mapping successfully finished!


---
### Timeseries Extraction: Daily Area-Weighted Precipitation
Calculates the rigorous intersection area-weighted daily mean precipitation for each basin. This is essential for temporal modeling and validation.

In [14]:
# ====================================================================
# Cell 6 — Export Daily Area-Weighted Mean Precipitation to CSV
# ====================================================================
import pandas as pd
import geopandas as gpd

print("--- Calculating and Exporting Daily Area-Weighted Means ---")

# Loop through the three datasets defined in Cell 4
for product_name, paths in datasets_to_process.items():
    print(f"Processing {product_name}...")
    
    # -------------------------------------------------------------
    # 1. Load geometries and intersect to calculate static weights
    # -------------------------------------------------------------
    fp = gpd.read_file(paths["shp"])
    if fp.crs is None: 
        fp = fp.set_crs(CRS_WGS)
    fp_utm = fp.to_crs(CRS_UTM)
    fp_utm["geometry"] = fp_utm.geometry.buffer(0)
    
    # Prepare basin boundaries
    ca_basin_use = ca_utm[["geometry"]].copy()
    ca_basin_use["geometry"] = ca_basin_use.geometry.buffer(0)
    ar_basin_use = ar_utm[["geometry"]].copy()
    ar_basin_use["geometry"] = ar_basin_use.geometry.buffer(0)
    
    # Intersections
    ca_inter = gpd.overlay(fp_utm, ca_basin_use, how="intersection", keep_geom_type=True)
    ar_inter = gpd.overlay(fp_utm, ar_basin_use, how="intersection", keep_geom_type=True)
    
    # Calculate weights (area fractions)
    ca_inter["w_frac_CA"] = ca_inter.geometry.area / ca_utm.geometry.area.sum()
    ar_inter["w_frac_AR"] = ar_inter.geometry.area / ar_utm.geometry.area.sum()
    
    # Extract only the weight mapping (pixel_id -> weight)
    w_ca = ca_inter[["pixel_id", "w_frac_CA"]].copy()
    w_ar = ar_inter[["pixel_id", "w_frac_AR"]].copy()
    
    # -------------------------------------------------------------
    # 2. Load Daily CSV and merge weights
    # -------------------------------------------------------------
    df = pd.read_csv(paths["csv"])
    df["Date"] = pd.to_datetime(df["Date"])
    df["Year"] = df["Date"].dt.year
    df = df[df["Year"].isin(YEARS)].copy()
    
    # Merge weights into the daily data
    df = df.merge(w_ca, on="pixel_id", how="left")
    df = df.merge(w_ar, on="pixel_id", how="left")
    
    # Fill NaNs with 0 for pixels that don't overlap a specific basin
    df["w_frac_CA"] = df["w_frac_CA"].fillna(0)
    df["w_frac_AR"] = df["w_frac_AR"].fillna(0)
    
    # -------------------------------------------------------------
    # 3. Calculate weighted daily precipitation
    # -------------------------------------------------------------
    df["P_weighted_CA"] = df["P_mm"] * df["w_frac_CA"]
    df["P_weighted_AR"] = df["P_mm"] * df["w_frac_AR"]
    
    # Group by Date to sum up the weighted values across all pixels
    df_daily_basin = df.groupby("Date")[["P_weighted_CA", "P_weighted_AR"]].sum().reset_index()
    
    # Rename columns for clarity in the final CSV
    df_daily_basin.rename(columns={
        "P_weighted_CA": "CA_P_mm",
        "P_weighted_AR": "AR_P_mm"
    }, inplace=True)
    
    # -------------------------------------------------------------
    # 4. Export to CSV
    # -------------------------------------------------------------
    out_csv = OUT_DIR / f"{product_name}_daily_area_weighted_mean_{YEARS[0]}_{YEARS[1]}.csv"
    df_daily_basin.to_csv(out_csv, index=False)
    print(f"  -> Saved to {out_csv.name}")

print("\n All daily timeseries successfully exported!")

--- Calculating and Exporting Daily Area-Weighted Means ---
Processing DAYMET...
  -> Saved to DAYMET_daily_area_weighted_mean_2017_2018.csv
Processing PRISM...
  -> Saved to PRISM_daily_area_weighted_mean_2017_2018.csv
Processing gridMET...
  -> Saved to gridMET_daily_area_weighted_mean_2017_2018.csv

 All daily timeseries successfully exported!


---
### Data Integrity Check: Spatial vs. Temporal Cross-Validation
Performs a sanity check. Mathematical conservation of mass is proven if the sum of weights strictly equals 1.0, and if the annual sums calculated from daily CSVs perfectly match the spatial outputs.

In [15]:
# ====================================================================
# Cell 6 — Data Integrity Check: Spatial vs. Temporal Cross-Validation
# ====================================================================

print("--- Sanity Check: Area Fractions and Annual Sums ---\n")

for product_name, paths in datasets_to_process.items():
    # Verify Fraction Sums (Spatial accuracy)
    fp = gpd.read_file(paths["shp"])
    if fp.crs is None: fp = fp.set_crs(CRS_WGS)
    fp_utm = fp.to_crs(CRS_UTM)
    fp_utm["geometry"] = fp_utm.geometry.buffer(0)
    
    ca_b = ca_utm[["geometry"]].copy(); ca_b["geometry"] = ca_b.geometry.buffer(0)
    ar_b = ar_utm[["geometry"]].copy(); ar_b["geometry"] = ar_b.geometry.buffer(0)
    
    ca_inter = gpd.overlay(fp_utm, ca_b, how="intersection", keep_geom_type=True)
    ar_inter = gpd.overlay(fp_utm, ar_b, how="intersection", keep_geom_type=True)
    
    ca_w_sum = (ca_inter.geometry.area / ca_utm.geometry.area.sum()).sum()
    ar_w_sum = (ar_inter.geometry.area / ar_utm.geometry.area.sum()).sum()
    
    print(f"[{product_name}] Weight Verifications (Expected 1.000000):")
    print(f"  CA sum(w_frac) = {ca_w_sum:.6f}")
    print(f"  AR sum(w_frac) = {ar_w_sum:.6f}")
    
    # Verify Daily to Annual Rollups (Temporal accuracy)
    csv_path = OUT_DIR / f"{product_name}_daily_area_weighted_mean_{YEARS[0]}_{YEARS[1]}.csv"
    if csv_path.exists():
        df = pd.read_csv(csv_path)
        df["Date"] = pd.to_datetime(df["Date"])
        df["Year"] = df["Date"].dt.year
        annual_sums = df.groupby("Year")[["CA_P_mm", "AR_P_mm"]].sum()
        
        print(f"[{product_name}] CSV Annual Sum Verifications:")
        for year in YEARS:
            if year in annual_sums.index:
                ca_ann = annual_sums.loc[year, "CA_P_mm"]
                ar_ann = annual_sums.loc[year, "AR_P_mm"]
                print(f"  {year} -> CA: {ca_ann:.0f} mm | AR: {ar_ann:.0f} mm")
    print("-" * 50)

--- Sanity Check: Area Fractions and Annual Sums ---

[DAYMET] Weight Verifications (Expected 1.000000):
  CA sum(w_frac) = 1.000000
  AR sum(w_frac) = 1.000000
[DAYMET] CSV Annual Sum Verifications:
  2017 -> CA: 1918 mm | AR: 1756 mm
  2018 -> CA: 2435 mm | AR: 2298 mm
--------------------------------------------------
[PRISM] Weight Verifications (Expected 1.000000):
  CA sum(w_frac) = 1.000000
  AR sum(w_frac) = 1.000000
[PRISM] CSV Annual Sum Verifications:
  2017 -> CA: 1784 mm | AR: 1620 mm
  2018 -> CA: 2004 mm | AR: 1933 mm
--------------------------------------------------
[gridMET] Weight Verifications (Expected 1.000000):
  CA sum(w_frac) = 1.000000
  AR sum(w_frac) = 1.000000
[gridMET] CSV Annual Sum Verifications:
  2017 -> CA: 1873 mm | AR: 1626 mm
  2018 -> CA: 2080 mm | AR: 1925 mm
--------------------------------------------------


---
### Validation Analytics: Multi-Source Timeseries Comparison
Utilizes the strictly area-weighted CSVs to compare remote sensing/climate products against local hydrological baseline inputs (CS01) and point rain gauges.

In [16]:
# ====================================================================
# Cell 7 — Validation Analytics: Multi-Source Timeseries Comparison
# ====================================================================

def load_met_txt(name, filepath):
    if not os.path.exists(filepath):
        if "CS01" not in name: print(f"  [Warning] Local met file not found: {filepath}")
        return pd.DataFrame()
    try:
        df = pd.read_csv(filepath, sep=r'\s+', header=None, engine='python')
        df['Date'] = pd.to_datetime(df[0], format='%m/%d/%Y-%H:%M')
        df['P_mm'] = df[6] * 1000.0
        df_daily = df.set_index('Date').resample('D').sum(numeric_only=True).reset_index()
        df_daily['Source'] = name 
        return df_daily[['Date', 'Source', 'P_mm']]
    except Exception as e:
        print(f"  [Error] Reading {name}: {e}")
        return pd.DataFrame()

def load_area_weighted_column(filepath, target_col, source_label):
    if not os.path.exists(filepath):
        print(f"  [Error] File not found: {filepath}")
        return pd.DataFrame()
    df = pd.read_csv(filepath)
    df['Date'] = pd.to_datetime(df['Date'])
    if target_col in df.columns:
        sub = df[['Date', target_col]].copy()
        sub.rename(columns={target_col: 'P_mm'}, inplace=True)
        sub['Source'] = source_label
        return sub
    return pd.DataFrame()

print("Initializing Time-Series Validation Sequence...")
df_cs01 = load_met_txt('CS01_Input', MET_FILES['CS01_Input'])

for ws_display_name, config in WS_MAPPING.items():
    print(f"\nProcessing Timeseries Analytics for: {ws_display_name}")
    data_list = []
    
    if not df_cs01.empty: data_list.append(df_cs01)
        
    g_key = config['gauge_key']
    if g_key in MET_FILES:
        local_name = f"Local ({g_key.replace('Gauge_', '')})"
        df_gauge = load_met_txt(local_name, MET_FILES[g_key])
        if not df_gauge.empty: data_list.append(df_gauge)
            
    target_col = config['csv_col']
    sub_daymet = load_area_weighted_column(OUT_DIR / f"DAYMET_daily_area_weighted_mean_{YEARS[0]}_{YEARS[1]}.csv", target_col, 'Daymet')
    sub_prism = load_area_weighted_column(OUT_DIR / f"PRISM_daily_area_weighted_mean_{YEARS[0]}_{YEARS[1]}.csv", target_col, 'PRISM (Climate)')
    sub_gridmet = load_area_weighted_column(OUT_DIR / f"gridMET_daily_area_weighted_mean_{YEARS[0]}_{YEARS[1]}.csv", target_col, 'GridMET (Radar)')
    
    if not sub_daymet.empty: data_list.append(sub_daymet)
    if not sub_prism.empty: data_list.append(sub_prism)
    if not sub_gridmet.empty: data_list.append(sub_gridmet)
        
    df_ws = pd.concat(data_list, ignore_index=True)
    df_ws = df_ws[(df_ws['Date'] >= START_DATE) & (df_ws['Date'] <= END_DATE)]
    df_pivot = df_ws.pivot_table(index='Date', columns='Source', values='P_mm', aggfunc='sum')
    
    # -------------------------------------------------------------
    # Visualization
    # -------------------------------------------------------------
    colors = {
        'CS01_Input': 'black',
        f"Local ({config['gauge_key'].replace('Gauge_', '')})": '#d62728', 
        'Daymet': '#1f77b4',          
        'PRISM (Climate)': '#2ca02c', 
        'GridMET (Radar)': '#9467bd'  
    }
    safe_name = ws_display_name.replace(" ", "_").replace("(", "").replace(")", "")
    cols_sorted = sorted(df_pivot.columns, key=lambda x: 0 if 'Daymet' in x else 1 if 'PRISM' in x else 2 if 'GridMET' in x else 3 if 'Local' in x else 4)
    
    # Plot 1: Cumulative
    plt.figure(figsize=(12, 7))
    df_cumsum = df_pivot.cumsum()
    for col in cols_sorted:
        c = colors.get(col, 'gray')
        ls, lw = ('-', 3.0) if ('Local' in col or 'CS01' in col) else ('--', 2.0)
        final_v = df_cumsum[col].iloc[-1]
        plt.plot(df_cumsum.index, df_cumsum[col], label=f"{col} ({final_v:.0f})", color=c, linestyle=ls, linewidth=lw, alpha=0.9)
    plt.title(f'{ws_display_name}: Cumulative Precipitation (2017-2018)', fontsize=16)
    plt.ylabel('Cumulative Precipitation (mm)', fontsize=14)
    plt.legend(title='Total (mm)', fontsize=11, loc='upper left')
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.savefig(OUT_DIR / f'Cumulative_{safe_name}.pdf')
    plt.savefig(OUT_DIR / f'Cumulative_{safe_name}.png')
    
    # Plot 2: Annual Totals
    # Note: using 'YE' for Pandas >= 2.2.0 compatibility
    df_annual = df_pivot.resample('YE').sum() 
    df_melt = df_annual.reset_index().melt(id_vars='Date', var_name='Source', value_name='Total_mm')
    df_melt['Year'] = df_melt['Date'].dt.year.astype(str)
    
    plt.figure(figsize=(10, 6))
    ax = sns.barplot(data=df_melt, x='Year', y='Total_mm', hue='Source', palette=colors)
    plt.title(f'{ws_display_name}: Annual Total Precipitation', fontsize=16)
    plt.ylabel('Total Precipitation (mm)', fontsize=14)
    plt.xlabel('Year', fontsize=14)
    plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', fontsize=11)
    
    for p in ax.patches:
        height = p.get_height()
        if height > 0:
            ax.annotate(f'{height:.0f}', (p.get_x() + p.get_width() / 2., height), 
                        ha='center', va='bottom', fontsize=10, fontweight='bold', 
                        rotation=0, xytext=(0, 5), textcoords='offset points')
    plt.ylim(0, df_melt['Total_mm'].max() * 1.15) 
    plt.savefig(OUT_DIR / f'Annual_{safe_name}.pdf')
    plt.savefig(OUT_DIR / f'Annual_{safe_name}.png')

    # Plot 3: Monthly Trends
    df_monthly = df_pivot.resample('ME').sum()
    plt.figure(figsize=(14, 6))
    for col in cols_sorted:
        c = colors.get(col, 'gray')
        ls, lw = ('-', 2.5) if ('Local' in col or 'CS01' in col) else ('--', 1.5)
        plt.plot(df_monthly.index, df_monthly[col], label=col, color=c, linestyle=ls, linewidth=lw)
    plt.title(f'{ws_display_name}: Monthly Precipitation Trends', fontsize=16)
    plt.ylabel('Monthly Precipitation (mm)', fontsize=14)
    plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', fontsize=11)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=2))
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    plt.xticks(rotation=45)
    plt.savefig(OUT_DIR / f'Monthly_{safe_name}.pdf')
    plt.savefig(OUT_DIR / f'Monthly_{safe_name}.png')
    plt.close('all')

print(f"\n All timeseries analytics exported successfully to: {OUT_DIR}")

Initializing Time-Series Validation Sequence...

Processing Timeseries Analytics for: Camp Branch (CA)

Processing Timeseries Analytics for: Arrowwood (AR)

 All timeseries analytics exported successfully to: /Users/benthosyy/Desktop/NEXRAD/Grid2WS/2_local_processing/Grid2WS_Validation_Output
